# Sparse MoE COD Training - 416px Resolution

This notebook trains the CamoXpert model with Sparse Mixture-of-Experts routing.

**Key Features:**
- ✅ Learned expert routing (router selects best experts per image)
- ✅ Anti-collapse system (adaptive coefficient + entropy regularization)
- ✅ 416px high resolution training
- ✅ 35-40% faster than dense experts
- 🎯 Target: IoU 0.75-0.76 (5-6% above SOTA)

**Expected Timeline:** ~6.8 hours (408 minutes)

## 1. Environment Setup & GPU Check

In [ ]:
import os
import torch
import json
from pathlib import Path

# Check GPU availability
print("="*70)
print("GPU CONFIGURATION")
print("="*70)

n_gpus = torch.cuda.device_count()
print(f"\n✅ Number of GPUs: {n_gpus}")

if n_gpus == 0:
    print("❌ ERROR: No GPUs detected!")
    raise RuntimeError("This notebook requires GPU acceleration")

for i in range(n_gpus):
    gpu_name = torch.cuda.get_device_name(i)
    gpu_memory = torch.cuda.get_device_properties(i).total_memory / 1e9
    print(f"   GPU {i}: {gpu_name} ({gpu_memory:.1f} GB)")

print(f"\n✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA version: {torch.version.cuda}")
print(f"✅ cuDNN version: {torch.backends.cudnn.version()}")

# Set CUDA environment variables for optimal performance
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'
os.environ['TORCH_USE_CUDA_DSA'] = '0'

print("\n" + "="*70)
print("READY TO TRAIN")
print("="*70)

## 2. Verify Dataset Path

In [ ]:
# Verify dataset exists
dataset_path = Path("/kaggle/input/cod10k-dataset/COD10K-v3")

print("="*70)
print("DATASET VERIFICATION")
print("="*70)

if dataset_path.exists():
    print(f"\n✅ Dataset found at: {dataset_path}")
    
    # Count images
    train_imgs = list((dataset_path / "Train" / "Image").glob("*.jpg"))
    test_imgs = list((dataset_path / "Test" / "Image").glob("*.jpg"))
    
    print(f"   Training images: {len(train_imgs)}")
    print(f"   Test images: {len(test_imgs)}")
    
    # Check structure
    required_dirs = ["Train/Image", "Train/GT", "Test/Image", "Test/GT"]
    for dir_name in required_dirs:
        dir_path = dataset_path / dir_name
        if dir_path.exists():
            print(f"   ✅ {dir_name}/")
        else:
            print(f"   ❌ {dir_name}/ - MISSING!")
else:
    print(f"\n❌ ERROR: Dataset not found at {dataset_path}")
    print("   Please check the dataset path in Kaggle input")
    raise FileNotFoundError(f"Dataset not found at {dataset_path}")

print("\n" + "="*70)

## 3. Create Checkpoint Directory

In [ ]:
# Create checkpoint directory
checkpoint_dir = Path("/kaggle/working/checkpoints_sparse_moe")
checkpoint_dir.mkdir(parents=True, exist_ok=True)

print(f"✅ Checkpoint directory: {checkpoint_dir}")
print(f"   Directory exists: {checkpoint_dir.exists()}")

## 4. Training Configuration

**Architecture:**
- Sparse MoE with 6 experts, top-2 selection (33% sparsity)
- EdgeNeXt-Base backbone
- 416px high resolution

**Anti-Collapse Measures:**
- Adaptive load balance coefficient: 0.00001 → 0.0005
- Entropy regularization: coefficient 0.001
- Real-time collapse detection

**Training Strategy:**
- Stage 1 (Epochs 0-40): Frozen backbone, batch size 12 per GPU
- Stage 2 (Epochs 41-200): Unfrozen backbone, batch size 8 per GPU
- Total time: ~6.8 hours

In [ ]:
# Display configuration
config = {
    "Model": "CamoXpert Sparse MoE",
    "Backbone": "EdgeNeXt-Base",
    "Resolution": "416px",
    "MoE Experts": 6,
    "Top-k Selection": 2,
    "Sparsity": "33% (2/6 experts active)",
    "Batch Size Stage 1": "12 per GPU (24 total with 2 GPUs)",
    "Batch Size Stage 2": "8 per GPU (16 total with 2 GPUs)",
    "Gradient Accumulation": 2,
    "Total Epochs": 200,
    "Stage 1 Epochs": 40,
    "Learning Rate": "0.0008 (stage 1), 0.0006 (stage 2)",
    "Scheduler": "Cosine Annealing",
    "Expected Time": "~6.8 hours",
    "Target IoU": "0.75-0.76"
}

print("="*70)
print("TRAINING CONFIGURATION")
print("="*70)
for key, value in config.items():
    print(f"{key:25s}: {value}")
print("="*70)

## 5. Launch Training

**What to expect:**

**Epoch 1-20 (Router Warmup):**
- IoU: 0.30 → 0.58
- Load balance coefficient: 0.00001 → 0.0005 (gradual increase)
- Router learning basic patterns, gradient explosion prevented

**Epoch 21-40 (Stage 1 Complete):**
- IoU: 0.58 → 0.62
- Full load balance pressure (0.0005)
- Router specialization emerging

**Epoch 41-200 (Stage 2):**
- IoU: 0.62 → 0.75-0.76
- Backbone unfrozen
- Expert specialization strengthens

**Router Health Monitoring:**
- Every epoch shows: `Router LB Loss: X.XXXXXX | Warmup: X.XX`
- Warnings if collapse detected (LB loss < 0.0001)

---

**Press Ctrl+C to stop training gracefully (checkpoint will be saved)**

In [ ]:
%%time

# Launch training with torchrun (DDP)
!torchrun --nproc_per_node=2 --master_port=29500 train_ultimate.py train \
    --use-ddp \
    --use-cod-specialized \
    --use-sparse-moe \
    --moe-num-experts 6 \
    --moe-top-k 2 \
    --dataset-path /kaggle/input/cod10k-dataset/COD10K-v3 \
    --checkpoint-dir /kaggle/working/checkpoints_sparse_moe \
    --backbone edgenext_base \
    --batch-size 12 \
    --stage2-batch-size 8 \
    --accumulation-steps 2 \
    --img-size 416 \
    --epochs 200 \
    --stage1-epochs 40 \
    --lr 0.0008 \
    --stage2-lr 0.0006 \
    --scheduler cosine \
    --min-lr 0.00001 \
    --warmup-epochs 5 \
    --deep-supervision \
    --num-workers 4

## 6. Load Training History & Results

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np

# Load history
history_path = checkpoint_dir / "history.json"

if history_path.exists():
    with open(history_path, 'r') as f:
        history = json.load(f)
    
    print("="*70)
    print("TRAINING RESULTS")
    print("="*70)
    
    # Extract metrics
    epochs = [h['epoch'] for h in history]
    train_loss = [h['train_loss'] for h in history]
    iou = [h['IoU'] for h in history]
    dice = [h['Dice_Score'] for h in history]
    
    # Find best epoch
    best_idx = np.argmax(iou)
    best_epoch = epochs[best_idx]
    best_iou = iou[best_idx]
    best_dice = dice[best_idx]
    
    print(f"\n🏆 BEST MODEL:")
    print(f"   Epoch: {best_epoch}")
    print(f"   IoU: {best_iou:.4f}")
    print(f"   Dice: {best_dice:.4f}")
    
    # Final epoch
    final_iou = iou[-1]
    final_dice = dice[-1]
    
    print(f"\n📊 FINAL EPOCH ({epochs[-1]}):")
    print(f"   IoU: {final_iou:.4f}")
    print(f"   Dice: {final_dice:.4f}")
    
    # Compare to SOTA
    sota_iou = 0.716
    improvement = (best_iou - sota_iou) / sota_iou * 100
    
    print(f"\n🎯 COMPARISON TO SOTA:")
    print(f"   SOTA COD10K IoU: {sota_iou:.4f}")
    print(f"   Your Best IoU: {best_iou:.4f}")
    print(f"   Improvement: {improvement:+.2f}%")
    
    if best_iou >= 0.75:
        print("\n   ✅ TARGET ACHIEVED! IoU ≥ 0.75")
    elif best_iou >= 0.74:
        print("\n   ✅ EXCELLENT! IoU ≥ 0.74 (close to target)")
    elif best_iou >= 0.72:
        print("\n   ✅ GOOD! IoU ≥ 0.72 (above SOTA)")
    
    print("\n" + "="*70)
    
else:
    print("❌ Training history not found. Training may not have completed.")

## 7. Visualize Training Progress

In [ ]:
if history_path.exists():
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Plot 1: IoU over epochs
    ax1 = axes[0, 0]
    ax1.plot(epochs, iou, 'b-', linewidth=2, label='IoU')
    ax1.axhline(y=0.716, color='r', linestyle='--', label='SOTA (0.716)', linewidth=2)
    ax1.axhline(y=0.75, color='g', linestyle='--', label='Target (0.75)', linewidth=2)
    ax1.axvline(x=40, color='orange', linestyle=':', label='Stage 1→2', linewidth=2)
    ax1.axvline(x=20, color='purple', linestyle=':', label='Warmup End', linewidth=1.5)
    ax1.scatter([best_epoch], [best_iou], color='gold', s=200, zorder=5, 
                marker='*', edgecolors='black', linewidths=2, label=f'Best ({best_iou:.4f})')
    ax1.set_xlabel('Epoch', fontsize=12)
    ax1.set_ylabel('IoU', fontsize=12)
    ax1.set_title('IoU Progress', fontsize=14, fontweight='bold')
    ax1.legend(fontsize=10)
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Dice Score over epochs
    ax2 = axes[0, 1]
    ax2.plot(epochs, dice, 'g-', linewidth=2, label='Dice Score')
    ax2.axvline(x=40, color='orange', linestyle=':', label='Stage 1→2', linewidth=2)
    ax2.axvline(x=20, color='purple', linestyle=':', label='Warmup End', linewidth=1.5)
    ax2.scatter([best_epoch], [best_dice], color='gold', s=200, zorder=5,
                marker='*', edgecolors='black', linewidths=2, label=f'Best ({best_dice:.4f})')
    ax2.set_xlabel('Epoch', fontsize=12)
    ax2.set_ylabel('Dice Score', fontsize=12)
    ax2.set_title('Dice Score Progress', fontsize=14, fontweight='bold')
    ax2.legend(fontsize=10)
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: Training Loss
    ax3 = axes[1, 0]
    ax3.plot(epochs, train_loss, 'r-', linewidth=2, label='Training Loss')
    ax3.axvline(x=40, color='orange', linestyle=':', label='Stage 1→2', linewidth=2)
    ax3.axvline(x=20, color='purple', linestyle=':', label='Warmup End', linewidth=1.5)
    ax3.set_xlabel('Epoch', fontsize=12)
    ax3.set_ylabel('Loss', fontsize=12)
    ax3.set_title('Training Loss', fontsize=14, fontweight='bold')
    ax3.legend(fontsize=10)
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Stage comparison
    ax4 = axes[1, 1]
    stage1_epochs = [e for e in epochs if e < 40]
    stage2_epochs = [e for e in epochs if e >= 40]
    stage1_iou = [iou[i] for i, e in enumerate(epochs) if e < 40]
    stage2_iou = [iou[i] for i, e in enumerate(epochs) if e >= 40]
    
    if stage1_epochs:
        ax4.plot(stage1_epochs, stage1_iou, 'b-', linewidth=3, label='Stage 1 (Frozen Backbone)')
    if stage2_epochs:
        ax4.plot(stage2_epochs, stage2_iou, 'g-', linewidth=3, label='Stage 2 (Unfrozen Backbone)')
    
    ax4.axhline(y=0.716, color='r', linestyle='--', label='SOTA', linewidth=2)
    ax4.set_xlabel('Epoch', fontsize=12)
    ax4.set_ylabel('IoU', fontsize=12)
    ax4.set_title('Stage Comparison', fontsize=14, fontweight='bold')
    ax4.legend(fontsize=10)
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(checkpoint_dir / 'training_progress.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\n✅ Plot saved to: {checkpoint_dir / 'training_progress.png'}")
else:
    print("❌ Cannot plot: training history not found")

## 8. Router Specialization Analysis (Optional)

This section analyzes whether the router learned to specialize experts for different camouflage types.

In [ ]:
# Check if router specialization analysis is needed
print("="*70)
print("ROUTER SPECIALIZATION ANALYSIS")
print("="*70)

print("\nTo analyze router specialization, you would need to:")
print("1. Load the best model checkpoint")
print("2. Run inference on a batch of images")
print("3. Extract routing probabilities from the router")
print("4. Analyze which experts are selected for different image types")

print("\n💡 Expected Specialization Pattern:")
print("   Forest camouflage     → Edge + Texture experts")
print("   Desert camouflage     → Texture + Contrast experts")
print("   Underwater camouflage → Frequency + Contrast experts")

print("\n📊 Router Health Indicators:")
if history_path.exists():
    print("   Check training logs for:")
    print("   - Router LB Loss at epoch 40: Should be 0.0002-0.0004")
    print("   - Router LB Loss at epoch 200: Should be 0.0001-0.0003")
    print("   - If LB loss < 0.0001: Potential router collapse")
    print("   - If LB loss > 0.01: Router instability")

print("\n" + "="*70)

## 9. Save Checkpoint Information

In [ ]:
# List checkpoint files
print("="*70)
print("CHECKPOINT FILES")
print("="*70)

checkpoint_files = list(checkpoint_dir.glob("*.pth"))

if checkpoint_files:
    print(f"\n✅ Found {len(checkpoint_files)} checkpoint file(s):\n")
    for ckpt in sorted(checkpoint_files):
        size_mb = ckpt.stat().st_size / 1e6
        print(f"   {ckpt.name:30s} ({size_mb:.1f} MB)")
    
    # Load best model info
    best_model_path = checkpoint_dir / "best_model.pth"
    if best_model_path.exists():
        checkpoint = torch.load(best_model_path, map_location='cpu')
        print(f"\n📦 BEST MODEL CHECKPOINT:")
        print(f"   Path: {best_model_path}")
        print(f"   Epoch: {checkpoint.get('epoch', 'N/A')}")
        print(f"   Best IoU: {checkpoint.get('best_iou', 'N/A'):.4f}")
        print(f"   Size: {best_model_path.stat().st_size / 1e6:.1f} MB")
        print(f"\n   Use this checkpoint for inference and deployment!")
else:
    print("\n⚠️  No checkpoint files found")

# Check history file
if history_path.exists():
    history_size = history_path.stat().st_size / 1e3
    print(f"\n✅ Training history: {history_path.name} ({history_size:.1f} KB)")

print("\n" + "="*70)

## 10. Summary & Next Steps

In [ ]:
print("="*70)
print("TRAINING SUMMARY")
print("="*70)

if history_path.exists():
    print(f"\n✅ Training completed successfully!")
    print(f"\n📊 Results:")
    print(f"   Total Epochs: {epochs[-1]}")
    print(f"   Best IoU: {best_iou:.4f} (Epoch {best_epoch})")
    print(f"   Best Dice: {best_dice:.4f}")
    print(f"   Final IoU: {final_iou:.4f}")
    
    # Comparison
    print(f"\n🎯 Performance vs SOTA:")
    print(f"   SOTA (COD10K): 0.716")
    print(f"   Your model: {best_iou:.4f}")
    print(f"   Improvement: {improvement:+.2f}%")
    
    # Interpretation
    if best_iou >= 0.76:
        print(f"\n   🌟 OUTSTANDING! IoU ≥ 0.76 (8%+ above SOTA)")
        print(f"   Router specialization worked excellently!")
    elif best_iou >= 0.75:
        print(f"\n   ✅ EXCELLENT! Target achieved (IoU ≥ 0.75)")
        print(f"   Router learned distinct expert patterns!")
    elif best_iou >= 0.74:
        print(f"\n   ✅ VERY GOOD! Close to target (IoU ≥ 0.74)")
        print(f"   Router specialization likely working")
    elif best_iou >= 0.72:
        print(f"\n   ✅ GOOD! Above SOTA baseline")
        print(f"   Consider checking router specialization logs")
    
    print(f"\n📁 Outputs:")
    print(f"   Best model: {checkpoint_dir / 'best_model.pth'}")
    print(f"   Training plot: {checkpoint_dir / 'training_progress.png'}")
    print(f"   History: {checkpoint_dir / 'history.json'}")
    
    print(f"\n🚀 Next Steps:")
    print(f"   1. Download best_model.pth for deployment")
    print(f"   2. Run inference on test set for final evaluation")
    print(f"   3. Optional: Apply test-time augmentation (TTA) for +0.5-1% IoU")
    print(f"   4. Optional: Ensemble with dense model for +1-2% IoU")
    
    if best_iou < 0.77:
        print(f"\n💡 To reach IoU 0.77-0.78:")
        print(f"   - Test-time augmentation (flips, scales)")
        print(f"   - Model ensemble (sparse + dense)")
        print(f"   - Extended training (300 epochs)")
        print(f"   - Higher resolution (512px if memory allows)")
else:
    print(f"\n⚠️  Training appears incomplete or history file missing")
    print(f"   Check training logs for any errors")

print("\n" + "="*70)
print("NOTEBOOK COMPLETE")
print("="*70)